In [ ]:
# Importando as bibliotecas necessárias.

import os
import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import tensorflow_datasets.public_api as tfds

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

import matplotlib.pyplot as plt

# Define o backend do Keras para o TensorFlow.
os.environ["KERAS_BACKEND"] = "tensorflow"


In [3]:
# Inicializa uma lista para armazenar os pares de textos
text_pairs = []

# Abre o arquivo "data.tsv" no modo de leitura com codificação UTF-8
with open("data.tsv", "r", encoding="utf-8") as f:
    # Itera por cada linha do arquivo
    for line in f:
        # Remove espaços extras e divide a linha utilizando a tabulação como separador
        fields = line.strip().split("\t")
        # Garante que a linha possui ao menos 4 campos (índices 0 a 3)
        if len(fields) < 4:
            continue
        french = fields[1]  # Obtém a segunda coluna (texto em francês)
        portuguese = "[start] " + fields[3] + " [end]"  # Obtém a quarta coluna (texto em português) e adiciona os marcadores de início e fim
        text_pairs.append((french, portuguese))

# Imprime o primeiro par de textos para verificação
print(text_pairs[0])


FileNotFoundError: [Errno 2] No such file or directory: 'data.tsv'

In [ ]:
# Itera 5 vezes para imprimir 5 pares de textos escolhidos aleatoriamente
for _ in range(5):
    # Seleciona um par de textos aleatório da lista 'text_pairs' e o imprime
    print(random.choice(text_pairs))


('Mais on peut aussi bien le concevoir comme un ensemble accolé.', '[start] Mas também é possível compreendê-lo como um todo. [end]')
("Je comprends qu'une erreur a été commise.", '[start] Eu entendo que um erro foi cometido. [end]')
('Comment épelle-t-on «\xa0pretty\xa0»\xa0?', '[start] Como se escreve "pretty"? [end]')
("Tôt ou tard, j'entrerai en possession de l'héritage de mes parents.", '[start] Cedo ou tarde, tomarei posse da herança dos meus pais. [end]')
('Nous étions heureux.', '[start] Éramos felizes. [end]')


In [ ]:
# Embaralha aleatoriamente os pares de textos
random.shuffle(text_pairs)

# Calcula o número de amostras para validação como 15% do total de pares
num_val_samples = int(0.15 * len(text_pairs))

# Define o número de amostras para treinamento, reservando 2 blocos de validação (um para validação e outro para teste)
num_train_samples = len(text_pairs) - 2 * num_val_samples

# Separa os pares para treinamento
train_pairs = text_pairs[:num_train_samples]

# Separa os pares para validação (logo após os pares de treinamento)
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]

# Separa os pares para teste (os pares restantes após treinamento e validação)
test_pairs = text_pairs[num_train_samples + num_val_samples :]

# Exibe a quantidade total de pares, bem como a divisão em treinamento, validação e teste
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")


33030 total pairs
23122 training pairs
4954 validation pairs
4954 test pairs


In [ ]:
# Define os caracteres de pontuação que serão removidos durante a padronização,
# adicionando os caracteres "«" e "»".
strip_chars = string.punctuation + "«" + "»"

# Remove os colchetes dos caracteres a serem removidos, mantendo-os no texto
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

# Define os parâmetros para o processamento dos textos
vocab_size = 25000       # Tamanho máximo do vocabulário
sequence_length = 20     # Comprimento máximo das sequências de saída
batch_size = 64          # Tamanho do lote (batch)
latentSpaceDimension = 32# Dimensão do espaço latente (latent space dimension).

def custom_standardization(input_string):
    """
    Função de padronização customizada para os textos.

    Converte o texto para minúsculas e remove os caracteres definidos em 'strip_chars'.
    """
    # Converte o texto para letras minúsculas
    lowercase = tf_strings.lower(input_string)
    # Remove os caracteres indesejados utilizando uma expressão regular
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

# Cria a camada de vetorização para os textos em francês
french_vectorization = TextVectorization(
    max_tokens=vocab_size,           # Número máximo de tokens
    output_mode="int",               # Saída como inteiros (índices dos tokens)
    output_sequence_length=sequence_length,  # Comprimento fixo das sequências
)

# Cria a camada de vetorização para os textos em português,
# utilizando a função de padronização customizada
portuguese_vectorization = TextVectorization(
    max_tokens=vocab_size,                # Número máximo de tokens
    output_mode="int",                    # Saída como inteiros (índices dos tokens)
    output_sequence_length=sequence_length + 1,  # Comprimento fixo das sequências (+1 para token especial)
    standardize=custom_standardization,   # Função customizada para padronização dos textos
)

# Extrai os textos em francês e português dos pares de treinamento
train_french_texts = [pair[0] for pair in train_pairs]      # Lista de textos em francês
train_portuguese_texts = [pair[1] for pair in train_pairs]    # Lista de textos em português

# Adapta as camadas de vetorização aos textos de treinamento
french_vectorization.adapt(train_french_texts)
portuguese_vectorization.adapt(train_portuguese_texts)


In [ ]:
def format_dataset(french, portuguese):
    # Aplica a vetorização nos textos em francês
    french = french_vectorization(french)
    # Aplica a vetorização nos textos em português
    portuguese = portuguese_vectorization(portuguese)
    # Retorna um dicionário com as entradas para o codificador e decodificador,
    # além dos rótulos para o treinamento do decodificador.
    return (
        {
            "encoder_inputs": french,          # Entrada para o codificador
            "decoder_inputs": portuguese[:, :-1],  # Entrada para o decodificador (exclui o último token)
        },
        portuguese[:, 1:],  # Rótulos para o decodificador (exclui o primeiro token)
    )


def make_dataset(pairs):
    # Separa os textos em francês e português dos pares fornecidos
    french_texts, portuguese_texts = zip(*pairs)
    french_texts = list(french_texts)
    portuguese_texts = list(portuguese_texts)

    # Cria um dataset do TensorFlow a partir dos textos
    dataset = tf_data.Dataset.from_tensor_slices((french_texts, portuguese_texts))
    # Agrupa os exemplos em lotes (batch)
    dataset = dataset.batch(batch_size)
    # Aplica a formatação dos dados utilizando a função format_dataset
    dataset = dataset.map(format_dataset)
    # Armazena o dataset em cache, embaralha e pré-carrega os lotes para otimizar o desempenho
    return dataset.cache().shuffle(2048).prefetch(16)


# Cria os datasets de treinamento e validação a partir dos pares correspondentes
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


In [ ]:
# Itera sobre o primeiro lote do dataset de treinamento
for inputs, targets in train_ds.take(1):
    # Imprime a forma do tensor que contém as entradas para o codificador
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')

    # Imprime a forma do tensor que contém as entradas para o decodificador
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')

    # Imprime a forma do tensor que contém os alvos (saídas) para o treinamento do decodificador
    print(f"targets.shape: {targets.shape}")


inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [ ]:
# Imprime os 5 primeiros textos em francês dos dados de treinamento
print(train_french_texts[:5])

# Imprime os 5 primeiros textos em português dos dados de treinamento
print(train_portuguese_texts[:5])


['Nous avons embarqué.', "L'affinité et la communion de pensée entre ces deux âmes est comme si elles se connaissaient depuis d'autres vies.", 'Je me rappelais les jours sereins et toujours les mêmes que, peu de semaines auparavant, je passais encore près de Marie, sans même entrevoir dans l’avenir une autre possibilité que celle d’un bonheur éternel.', "Il est difficile de calculer les résultats de l'élection.", 'Les dames sont jouées de différentes manières, malgré leur utilisation répandue dans tous les pays civilisés.']
['[start] Nós embarcamos. [end]', '[start] A afinidade e a comunhão de pensamentos entre aquelas duas almas é tal, como se ambas se conhecessem desde outras vidas. [end]', '[start] Eu me lembrei daqueles dias calmos e sempre iguais, que, algumas semanas antes, ainda pudera passar ao lado de Maria, sem sequer vislumbrar no futuro senão a perspectiva de uma felicidade eterna. [end]', '[start] É difícil calcular os resultados da eleição. [end]', '[start] O jogo de dama

In [ ]:
# Supondo que a classe de atenção Bahdanau já esteja definida corretamente:
class BahdanauAttention(layers.Layer):
    def __init__(self, units, verbose=0):
        """
        Inicializa a camada de atenção Bahdanau.

        Args:
            units (int): Número de unidades para as camadas densas internas.
            verbose (int, opcional): Nível de detalhamento para debug. Padrão é 0.
        """
        super(BahdanauAttention, self).__init__()
        # Camadas densas para transformar a query e os valores
        self.W1 = layers.Dense(units)
        self.W2 = layers.Dense(units)
        # Camada densa para gerar os scores de atenção
        self.V = layers.Dense(1)
        self.verbose = verbose

    def call(self, query, values):
        """
        Executa a atenção Bahdanau.

        Args:
            query (Tensor): Vetor de consulta com shape (batch_size, hidden_size).
            values (Tensor): Valores com shape (batch_size, time_steps, hidden_size).

        Returns:
            context_vector (Tensor): Vetor de contexto com shape (batch_size, hidden_size).
            attention_weights (Tensor): Pesos de atenção com shape (batch_size, time_steps, 1).
        """
        # Adiciona uma dimensão temporal à query para permitir a soma com os valores.
        # Novo shape: (batch_size, 1, hidden_size)
        query_with_time_axis = tf.expand_dims(query, 1)

        # Calcula os scores de atenção combinando a query e os valores.
        # A função tanh é aplicada após as transformações lineares.
        # Resultado: (batch_size, time_steps, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # Normaliza os scores usando softmax para obter os pesos de atenção.
        # Shape: (batch_size, time_steps, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # Multiplica os pesos de atenção pelos valores para obter o vetor de contexto.
        # Shape intermediário: (batch_size, time_steps, hidden_size)
        context_vector = attention_weights * values

        # Reduz a soma ao longo do eixo temporal para combinar as informações.
        # Shape final: (batch_size, hidden_size)
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights


In [ ]:

# Camada customizada para aplicar a atenção para cada timestep do decodificador
class TimeDistributedAttention(layers.Layer):
    def __init__(self, attention_layer, **kwargs):
        """
        Inicializa a camada de atenção distribuída no tempo.

        Args:
            attention_layer (layers.Layer): Instância da camada de atenção a ser aplicada.
            **kwargs: Argumentos adicionais para a camada pai.
        """
        super(TimeDistributedAttention, self).__init__(**kwargs)
        self.attention_layer = attention_layer

    def call(self, inputs):
        """
        Aplica a atenção a cada timestep dos outputs do decodificador.

        Args:
            inputs (list): Lista contendo [decoder_outputs, encoder_outputs] onde:
                - decoder_outputs: (batch_size, dec_timesteps, latentSpaceDimension)
                - encoder_outputs: (batch_size, enc_timesteps, latentSpaceDimension)

        Returns:
            context_sequence (Tensor): Sequência de vetores de contexto para cada timestep do decodificador,
                                       com shape (batch_size, dec_timesteps, latentSpaceDimension).
        """
        # Desempacota as entradas: outputs do decodificador e do codificador
        decoder_outputs, encoder_outputs = inputs

        # Transpõe os outputs do decodificador para iterar sobre o eixo temporal.
        # Novo shape: (dec_timesteps, batch_size, latentSpaceDimension)
        decoder_outputs_transposed = tf.transpose(decoder_outputs, perm=[1, 0, 2])

        # Função que aplica a atenção para um único timestep do decodificador.
        def apply_attention(decoder_timestep):
            """
            Aplica a atenção para um timestep específico.

            Args:
                decoder_timestep (Tensor): Vetor do timestep do decodificador com shape (batch_size, latentSpaceDimension).

            Returns:
                context (Tensor): Vetor de contexto calculado para o timestep com shape (batch_size, latentSpaceDimension).
            """
            # Utiliza o vetor do timestep como query para calcular a atenção sobre os encoder_outputs.
            context, _ = self.attention_layer(decoder_timestep, encoder_outputs)
            return context

        # Aplica a função de atenção para cada timestep usando tf.map_fn.
        # O resultado possui shape: (dec_timesteps, batch_size, latentSpaceDimension)
        context_sequence_transposed = tf.map_fn(
            apply_attention,
            decoder_outputs_transposed,
            fn_output_signature=tf.float32
        )

        # Transpõe de volta para o formato original: (batch_size, dec_timesteps, latentSpaceDimension)
        context_sequence = tf.transpose(context_sequence_transposed, perm=[1, 0, 2])
        return context_sequence


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

class PositionalEmbedding(layers.Layer):
    """
    Camada de embedding posicional que soma os embeddings dos tokens e das posições.
    """

    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        """
        Inicializa a camada de embedding posicional.

        Args:
            sequence_length (int): Comprimento máximo da sequência.
            vocab_size (int): Tamanho do vocabulário.
            embed_dim (int): Dimensão dos embeddings.
            **kwargs: Argumentos adicionais para a classe base.
        """
        super().__init__(**kwargs)
        # Embedding para os tokens com dimensão de entrada igual ao tamanho do vocabulário
        # e dimensão de saída igual a embed_dim
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        # Embedding para as posições com dimensão de entrada igual ao comprimento da sequência
        # e dimensão de saída igual a embed_dim
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        """
        Soma os embeddings dos tokens e das posições.

        Args:
            inputs (Tensor): Tensores contendo índices dos tokens com shape
                             (batch_size, sequence_length).

        Returns:
            Tensor: Soma dos embeddings dos tokens e das posições com shape
                    (batch_size, sequence_length, embed_dim).
        """
        # Obtém os embeddings dos tokens: shape (batch_size, sequence_length, embed_dim)
        embedded_tokens = self.token_embeddings(inputs)

        # Cria um tensor constante com as posições: shape (sequence_length,)
        positions = tf.range(start=0, limit=self.sequence_length, delta=1)

        # Obtém os embeddings das posições: shape (sequence_length, embed_dim)
        embedded_positions = self.position_embeddings(positions)

        # Expande a dimensão para que os embeddings das posições possam ser somados
        # aos embeddings dos tokens em todas as amostras do batch.
        # Novo shape: (1, sequence_length, embed_dim)
        embedded_positions = tf.expand_dims(embedded_positions, axis=0)

        # Retorna a soma dos embeddings dos tokens com os embeddings das posições
        return embedded_tokens + embedded_positions

    def get_config(self):
        """
        Retorna a configuração da camada para possibilitar a serialização.

        Returns:
            dict: Dicionário de configuração da camada.
        """
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


In [ ]:
class AttentionLSTMCell(layers.Layer):
    """
    Célula LSTM com mecanismo de atenção integrada.
    Esta célula utiliza um mecanismo de atenção (attention_layer)
    para calcular um vetor de contexto a cada timestep, que é concatenado
    com o novo estado oculto da LSTM para formar a saída.
    """

    def __init__(self, units, attention_layer, **kwargs):
        """
        Inicializa a célula LSTM com atenção.

        Args:
            units (int): Número de unidades da LSTM.
            attention_layer (layers.Layer): Camada de atenção a ser utilizada.
            **kwargs: Argumentos adicionais.
        """
        super(AttentionLSTMCell, self).__init__(**kwargs)
        self.units = units  # Número de unidades para a LSTM
        self.attention_layer = attention_layer  # Mecanismo de atenção a ser aplicado
        self.lstm_cell = layers.LSTMCell(units)  # Instância da célula LSTM padrão

    @property
    def state_size(self):
        """
        Define o tamanho dos estados internos da célula.

        Retorna uma lista contendo:
          - hidden: vetor oculto com dimensão (units)
          - cell: vetor de célula com dimensão (units)
          - prev_attention: vetor de atenção do timestep anterior com dimensão (units)
          - encoder_outputs: estados do encoder com shape (None, units)
            (None indica que o número de timesteps do encoder pode variar)
        """
        return [self.units, self.units, self.units, tf.TensorShape([None, self.units])]

    @property
    def output_size(self):
        """
        Define o tamanho da saída da célula.

        A saída é a concatenação do novo estado oculto e do vetor de contexto,
        portanto, sua dimensão é a soma das dimensões dos dois vetores.
        """
        return self.units + self.units

    def call(self, inputs, states):
        """
        Executa um passo de tempo da célula LSTM com atenção.

        Args:
            inputs (Tensor): Embedding do token atual com shape (batch_size, embed_dim).
            states (list): Lista contendo os estados anteriores:
                [hidden, cell, prev_attention, encoder_outputs]

        Retorna:
            new_output (Tensor): Saída da célula com shape (batch_size, output_size),
                                 que é a concatenação do novo estado oculto e do vetor de contexto.
            new_state (list): Lista atualizada dos estados, mantendo os valores para:
                              [new_hidden, new_cell, context_vector, encoder_outputs].
        """
        # Desempacota os estados: hidden, cell, atenção anterior e outputs do encoder
        hidden, cell, prev_attention, encoder_outputs = states

        # Concatena o embedding do token atual com o vetor de atenção do timestep anterior (input feeding)
        input_combined = tf.concat([inputs, prev_attention], axis=-1)

        # Processa o input combinado pela LSTMCell para obter o novo estado oculto e o novo estado da célula
        output, [new_hidden, new_cell] = self.lstm_cell(input_combined, [hidden, cell])

        # Calcula a atenção utilizando o novo estado oculto como query e os outputs do encoder como values
        context_vector, attention_weights = self.attention_layer(new_hidden, encoder_outputs)

        # Define a saída da célula como a concatenação do novo estado oculto com o vetor de contexto
        new_output = tf.concat([new_hidden, context_vector], axis=-1)

        # Atualiza os estados:
        # - new_hidden: novo estado oculto da LSTM
        # - new_cell: novo estado da célula da LSTM
        # - context_vector: vetor de contexto obtido pela atenção (será usado no próximo timestep)
        # - encoder_outputs: permanece constante ao longo da decodificação
        new_state = [new_hidden, new_cell, context_vector, encoder_outputs]

        return new_output, new_state


In [ ]:
# Camada densa para adaptar os outputs do encoder para o espaço latente
encoder_adapter = layers.Dense(latentSpaceDimension, activation='relu', name='encoder_adapter')

# --- Parâmetros e camadas de embedding (assumindo que já estejam definidos) ---
# sequence_length: comprimento máximo da sequência
# vocab_size: tamanho do vocabulário
# embed_dim: dimensão dos embeddings dos tokens
# latentSpaceDimension: dimensão do espaço latente

# --- Encoder ---
# Entrada para o encoder: sequência de índices (tokens) com tamanho variável
encoder_inputs = Input(shape=(None,), dtype="int64", name="encoder_inputs")
# Aplica o embedding posicional nos tokens do encoder
encoder_embed = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# Camada LSTM do encoder que retorna tanto as sequências quanto os estados finais
encoder_lstm = layers.LSTM(latentSpaceDimension,
                           return_sequences=True,
                           return_state=True,
                           name='encoder_lstm')
# Processa a sequência de embeddings e obtém os outputs e os estados finais (hidden e cell)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embed)
# Armazena os estados finais do encoder para uso posterior no decoder
encoder_states = [state_h, state_c]

# Adapta os outputs do encoder para o espaço latente usando a camada densa definida anteriormente
adapted_encoder_outputs = encoder_adapter(encoder_outputs)

# --- Decoder ---
# Entrada para o decoder: sequência de índices (tokens) com tamanho variável
decoder_inputs = Input(shape=(None,), dtype="int64", name="decoder_inputs")
# Aplica a camada de embedding para os tokens do decoder
decoder_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(decoder_inputs)

# Cria o cell customizado de LSTM com atenção (input feeding) utilizando a AttentionLSTMCell
attention_cell = AttentionLSTMCell(latentSpaceDimension, attention_layer)
# Cria uma camada RNN que utiliza a célula customizada, retornando sequências e estados
decoder_rnn = layers.RNN(attention_cell, return_sequences=True, return_state=True)

# Função auxiliar para inicializar o vetor de atenção com zeros para cada exemplo do batch
def get_initial_attention(x):
    batch_size = tf.shape(x)[0]
    return tf.zeros((batch_size, latentSpaceDimension), dtype=tf.float32)

# Inicializa o vetor de atenção com zeros, utilizando o estado hidden do encoder como base para definir o batch size
initial_attention = layers.Lambda(get_initial_attention)(state_h)

# Define o estado inicial do decoder como a combinação dos estados do encoder e o vetor de atenção inicial:
# [hidden, cell, prev_attention, adapted_encoder_outputs]
initial_state = encoder_states + [initial_attention, adapted_encoder_outputs]

# Executa a RNN do decoder, processando os embeddings do decoder com o estado inicial definido
decoder_rnn_outputs = decoder_rnn(decoder_embeddings, initial_state=initial_state)
# A saída do decoder é a sequência resultante (primeiro elemento do retorno da RNN)
decoder_outputs = decoder_rnn_outputs[0]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'attention_lstm_cell_8', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [ ]:
# Camada densa final para gerar as predições a partir dos outputs do decoder.
decoder_dense = layers.Dense(vocab_size, activation='softmax', name='decoder_dense')
final_outputs = decoder_dense(decoder_outputs)

# --- Modelo Final ---
# Define o modelo sequencial que recebe as entradas do encoder e do decoder,
# e produz as predições finais.
model = Model([encoder_inputs, decoder_inputs], final_outputs, name='seq2seq_input_feeding')

# Compila o modelo utilizando o otimizador Adam, a função de perda
# de entropia cruzada esparsa e a métrica de acurácia.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Exibe um resumo da arquitetura do modelo.
model.summary()


Model: "seq2seq_input_feeding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_14   │ (None, 20, 128)        │      1,922,560 │ encoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_lstm (LSTM)       │ [(None, 20, 32),       │         20,608 │ positional_embedding_… │
│                           │ (None, 32), (None,     │                │                        │
│                           │ 32)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_38 (Embedding)  │ (None, None, 128)      │      1,920,000 │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_8 (Lambda)         │ (None, 32)             │              0 │ encoder_lstm[0][1]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_adapter (Dense)   │ (None, 20, 32)         │          1,056 │ encoder_lstm[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rnn_8 (RNN)               │ [(None, None, 64),     │          2,145 │ embedding_38[0][0],    │
│                           │ (None, 32), (None,     │                │ encoder_lstm[0][1],    │
│                           │ 32), (None, 32)]       │                │ encoder_lstm[0][2],    │
│                           │                        │                │ lambda_8[0][0],        │
│                           │                        │                │ encoder_adapter[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dense (Dense)     │ (None, None, 15000)    │        975,000 │ rnn_8[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,841,369 (18.47 MB)

 Trainable params: 4,841,369 (18.47 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# =============================================================================
# Preparação dos arrays para treinamento e teste a partir dos textos brutos
# =============================================================================

# --- Dados de Treinamento ---
# Converte os textos de treinamento para arrays utilizando as camadas de vetorização.
# O resultado é um array de inteiros para o encoder.
encoder_input_data = french_vectorization(np.array(train_french_texts)).numpy()

# Converte os textos de treinamento do português para arrays tokenizados.
# O resultado é um array onde cada linha representa uma sequência tokenizada.
portuguese_tokenized = portuguese_vectorization(np.array(train_portuguese_texts)).numpy()

# Para o decoder, definimos:
# - decoder_input_data: sequência de entrada do decoder (todos os tokens, exceto o último),
#   pois o modelo recebe essa sequência para prever o próximo token.
# - decoder_target_data: sequência alvo (todos os tokens, exceto o primeiro), que é o que o
#   modelo deve prever.
decoder_input_data = portuguese_tokenized[:, :-1]
decoder_target_data = portuguese_tokenized[:, 1:]

# --- Dados de Teste ---
# Extrai os textos de teste dos pares e converte utilizando as camadas de vetorização.

# Obtém os textos em francês e português dos pares de teste
test_french_texts = [pair[0] for pair in test_pairs]
test_portuguese_texts = [pair[1] for pair in test_pairs]

# Converte os textos de teste para arrays de inteiros
encoder_input_test = french_vectorization(np.array(test_french_texts)).numpy()
portuguese_tokenized_test = portuguese_vectorization(np.array(test_portuguese_texts)).numpy()

# Prepara as sequências para o decoder no teste, seguindo a mesma lógica aplicada aos dados de treinamento.
decoder_input_test = portuguese_tokenized_test[:, :-1]
decoder_target_test = portuguese_tokenized_test[:, 1:]

# =============================================================================
# Treinamento e Avaliação do Modelo com a Nova Arquitetura (com atenção)
# =============================================================================

# Treina o modelo utilizando os arrays preparados para o encoder e decoder.
history = model.fit(
    x=[encoder_input_data, decoder_input_data],
    y=decoder_target_data,
    batch_size=64,
    epochs=5,
    validation_data=([encoder_input_test, decoder_input_test], decoder_target_test)
)

# Avalia o modelo no conjunto de teste.
test_loss, test_accuracy = model.evaluate(
    x=[encoder_input_test, decoder_input_test],
    y=decoder_target_test
)

# Exibe os resultados da avaliação no teste.
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)


Epoch 1/5
362/362 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - accuracy: 0.6427 - loss: 4.8249 - val_accuracy: 0.7024 - val_loss: 2.1633
Epoch 2/5
362/362 ━━━━━━━━━━━━━━━━━━━━ 382s 1s/step - accuracy: 0.7024 - loss: 2.1212 - val_accuracy: 0.7146 - val_loss: 2.0174
Epoch 3/5
362/362 ━━━━━━━━━━━━━━━━━━━━ 394s 1s/step - accuracy: 0.7165 - loss: 1.9576 - val_accuracy: 0.7246 - val_loss: 1.9211
Epoch 4/5
362/362 ━━━━━━━━━━━━━━━━━━━━ 429s 1s/step - accuracy: 0.7297 - loss: 1.8318 - val_accuracy: 0.7366 - val_loss: 1.8310
Epoch 5/5
362/362 ━━━━━━━━━━━━━━━━━━━━ 387s 1s/step - accuracy: 0.7405 - loss: 1.7253 - val_accuracy: 0.7435 - val_loss: 1.7545
155/155 ━━━━━━━━━━━━━━━━━━━━ 35s 225ms/step - accuracy: 0.7407 - loss: 1.7767
Test loss: 1.7544641494750977
Test accuracy: 0.7434595823287964


In [ ]:
# Resume o histórico de acurácia: plota a acurácia para os conjuntos de treinamento e validação
plt.plot(history.history['accuracy'])       # Acurácia durante o treinamento
plt.plot(history.history['val_accuracy'])     # Acurácia durante a validação
plt.title(model.name + ' accuracy')           # Define o título do gráfico (nome do modelo + 'accuracy')
plt.ylabel('accuracy')                        # Rótulo do eixo y
plt.xlabel('epoch')                           # Rótulo do eixo x
plt.legend(['train', 'val'], loc='upper left')  # Adiciona a legenda indicando os conjuntos
plt.show()                                    # Exibe o gráfico de acurácia

# Resume o histórico de perda: plota a perda para os conjuntos de treinamento e validação
plt.plot(history.history['loss'])             # Perda durante o treinamento
plt.plot(history.history['val_loss'])           # Perda durante a validação
plt.title(model.name + ' loss')                # Define o título do gráfico (nome do modelo + 'loss')
plt.ylabel('loss')                            # Rótulo do eixo y
plt.xlabel('epoch')                           # Rótulo do eixo x
plt.legend(['train', 'val'], loc='upper left')  # Adiciona a legenda indicando os conjuntos
plt.show()                                    # Exibe o gráfico de perda
